# Landcover attributes extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the lancover characteristics from the Corine dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made avaialable in this repository due to redistribution and storage-space reasons.


## Requirements
**Python:**
* Python>=3.6
* Jupyter
* geopandas=0.10.2
* glob
* numpy
* os
* pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**
* data/gee/landcover/EStreams_lulc{1990, 2000, 2006, 2012, 2018}_attributes_gee.csv. Landcover attributes CSV-files exported from GEE
* data/shapefiles/estreams_catchments.shp

**Directory:**
* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* CORINE Land Cover — Copernicus Land Monitoring Service. European Environment Agency [data set], Copenhagen, Denmark https://land.copernicus.eu/en/products/corine-land-cover.

## Licenses
* Corine: Open access. "The Copernicus land monitoring products and services are made available on a principle of full, open and free access, as established by the Commission Delegated Regulation (EU) No 1159/2013 of 12 July 2013." https://land.copernicus.eu/en/data-policy (Last access 27 November 2023)

## Observations
* This notebook assumes that the GEE code to export the landcover descriptors from the Corine dataset (EStreams_landscape_attributes_landcover_gee.txt) was run before in the GEE platform and that all the output CSV-files are locally available. 
* It is possible that there are more than one CSV-file per year if the user decided to subset the catchments in smaller groups for optimze the exportation. 
* All the lulc csv-files must be placed in a single folder together. 

# Import modules

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm as tqdm
import glob
from utils.landcover import *

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.

In [ ]:
# Non-editable variables:
PATH_LULC="data/gee/landcover"
PATH_OUTPUT = "results/staticattributes/"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [ ]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries.head()

In [ ]:
print("The total number of catchments to be processed are:", len(catchment_boundaries))

# Reproject to projected coordinates system

In [ ]:
# Define the target CRS to ETRS89 LAEA
target_crs = 'EPSG:3035' 

# Reproject the GeoDataFrame to the target CRS
catchment_boundaries_reprojected = catchment_boundaries.to_crs(target_crs)

## GEE outputs

In [ ]:
# Check the files in the subdirectory:
filenames = glob.glob(PATH_LULC + "/*.csv")
filenames

In [ ]:
# First we create an empty dataframe for the data:
landcover_df = pd.DataFrame()

# Loop for reading and concatenating the data:
for file in tqdm.tqdm(filenames):
    
    # First we read our data:
    landcover_file = pd.read_csv(file)
    landcover_file.drop(["system:index", ".geo"], axis = 1, inplace = True)
    landcover_file["class_name"] = "lulc_" + landcover_file["year"].astype(str) + "_" + landcover_file["class"].astype(str)
    year = landcover_file.loc[0, "year"]
    
    # Here we can create a pivot-table to organize our dataset:
    landcover_pivot = pd.pivot_table(
        landcover_file,
        values='area_sqm',          
        index='code',               # Rows are based on 'code'
        columns='class_name',       # Columns are based on 'class_name'
        fill_value=np.nan)
    
    # Total are per year:
    #landcover_pivot[str(year)+"_tot_area"] = landcover_pivot.sum(axis = 1)
    #landcover_pivot.iloc[:, :-1] = landcover_pivot.iloc[:, :-1].div(landcover_pivot[str(year)+"_tot_area"], axis=0)
    landcover_pivot["tot_area_"+str(year)] = landcover_pivot.sum(axis = 1)
    landcover_pivot.iloc[:, :-1] = landcover_pivot.iloc[:, :-1].div(landcover_pivot["tot_area_"+str(year)], axis=0)
    
    # Now we proceed with the concatenation:
    landcover_df = pd.concat([landcover_df, landcover_pivot], axis=1)
    
    # Here we deal with the case we have more than one file for the same year:
    landcover_df = landcover_df.T.groupby(level=0).apply(lambda group: group.ffill().bfill().iloc[0]).T

In [ ]:
# Here we add the majority class for each basin:
landcover_df = pd.concat([landcover_df, landcover_df.apply(get_majority_columns, axis=1)], axis=1)

landcover_df

In [ ]:
# Here we add the percentage of each catchment area covered by the Corine (there are countries not covered)
columns_tot_areas = ["tot_area_1990", "tot_area_2000", "tot_area_2006", "tot_area_2012", "tot_area_2018"]

landcover_df.loc[:, columns_tot_areas] = landcover_df.loc[:, columns_tot_areas].div(catchment_boundaries_reprojected.set_index("basin_id").area, axis=0)
landcover_df

In [ ]:
# Here we sort the index:
landcover_df = landcover_df.sort_index(axis=0)
landcover_df.index.name = "basin_id"
landcover_df

In [ ]:
# Round the data to 3 decimals:
landcover_df.iloc[:, 0:-5] = landcover_df.iloc[:, 0:-5].astype(float).round(3)
landcover_df

# Data export

In [ ]:
# Export the final dataset:
landcover_df.to_csv(PATH_OUTPUT+"estreams_landcover_attributes.csv")

# End